In [89]:
import numpy as np 
import pandas as pd

df = pd.read_csv("iris.data")

In [90]:
df.columns = [
              'SepalLengthCm',
              'SepalWidthCm',
              'PetalLengthCm',
              'PetalWidthCm',
              'Species']

In [91]:
df.isnull().sum()

SepalLengthCm    0
SepalWidthCm     0
PetalLengthCm    0
PetalWidthCm     0
Species          0
dtype: int64

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
SepalLengthCm    149 non-null float64
SepalWidthCm     149 non-null float64
PetalLengthCm    149 non-null float64
PetalWidthCm     149 non-null float64
Species          149 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [97]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,149.000000,149.000000,149.000000,149.000000
mean,5.848322,3.051006,3.774497,1.205369
std,0.828594,0.433499,1.759652,0.761292
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [99]:

from sklearn.preprocessing import LabelBinarizer
species_lb = LabelBinarizer()
Y = species_lb.fit_transform(df.Species.values)

In [101]:
from sklearn.preprocessing import normalize
FEATURES = df.columns[0:4]
X_data = df[FEATURES].as_matrix()
X_data = normalize(X_data)

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y, test_size=0.3, random_state=1)
X_train.shape
X_test.shape

(45, 4)

In [104]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 100


In [105]:
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 128 # 1st layer number of neurons
n_input = X_train.shape[1] # input shape (105, 4)
n_classes = y_train.shape[1] # classes to predict

In [106]:
X = tf.placeholder("float", shape=[None, n_input])
y = tf.placeholder("float", shape=[None, n_classes])

# Dictionary of Weights and Biases
weights = {
  'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
  'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
  'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1])),
  'b2': tf.Variable(tf.random_normal([n_hidden_2])),
  'out': tf.Variable(tf.random_normal([n_classes]))
}

In [107]:
def forward_propagation(x):
    # Hidden layer1
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output fully connected layer
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out'] 
    return out_layer

In [108]:
yhat = forward_propagation(X)
ypredict = tf.argmax(yhat, axis=1)

In [109]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [110]:
init = tf.global_variables_initializer()

from datetime import datetime
startTime = datetime.now()

with tf.Session() as sess:
    sess.run(init)
    
    #writer.add_graph(sess.graph)
    #EPOCHS
    for epoch in range(training_epochs):
        #Stochasting Gradient Descent
        for i in range(len(X_train)):
            summary = sess.run(train_op, feed_dict={X: X_train[i: i + 1], y: y_train[i: i + 1]})
        
        train_accuracy = np.mean(np.argmax(y_train, axis=1) == sess.run(ypredict, feed_dict={X: X_train, y: y_train}))
        test_accuracy  = np.mean(np.argmax(y_test, axis=1) == sess.run(ypredict, feed_dict={X: X_test, y: y_test}))
                
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
        #print("Epoch = %d, train accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy))

    sess.close()
print("Time taken:", datetime.now() - startTime)

Epoch = 1, train accuracy = 67.31%, test accuracy = 64.44%
Epoch = 2, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 3, train accuracy = 95.19%, test accuracy = 97.78%
Epoch = 4, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 5, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 6, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 7, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 8, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 9, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 10, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 11, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 12, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 13, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 14, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 15, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 16, train accuracy = 96.15%, test accuracy = 97.78%
Epoch = 17, train accuracy = 96.15%, test accurac